# Example Usage for Echo Robot 

The [Echo](https://www.beckman.com/liquid-handlers/echo-650-series) is an acoustic liquid handler, used in biology experiments to move very small amounts of liquid from a source [plate](https://en.wikipedia.org/wiki/Microplate) to a destination plate. In practice, the echo robot is frequently used to pipet potential drugs or other perturbagens to be tested into the experimental plate, diluting the compounds to test them at different strengths. Here, the solvent for dilution is assumed to be DMSO, the common drug solvent, but other solvent can be used. Because position within the plate can affect the experiment results, a typical use case is to randomize the position of each test compound, and to have multiple replicates within the experiment plate.

This colab is designed to start from the set of compound concentrations and replicates desired in the experiment and create a randomized plate format and the CSV file to instruct the Echo robot to create that experimental plate.

There are two styles of experiment that are supported.
1. **Dilution plate** : In this setup, the compounds in the source plate are all assumed to be at the same concentration, and the destination plate creates dilutions of these compounds using DMSO.
     * This colab demonstrates this style, using 'build_source_map' and 'create_echo_transfer_list' to set up the echo transfer information.
1. **No Dilutions** : In this setup, the source plate is assumed to already contain the exact compounds and concentrations needed for the destination plate. In this setup, it is assumed that the same volume will be used for each well in the destination plate.
     * In these cases, the echo is often used to add drug dosage to wells with existing volumes, so a dilution constant is used to convert from the concentrations on the source plate to the final concentration on the destination plate. (For example, if you are transferring 100 nl into a well that already has 49.9 ul, this is a 500-fold dilution, and a compound that should be 0.1 uM on the destination plate would be expected to be at 50 uM on the source plate.)
     * This approach is demonstrated in the other example colab, using 'build_source_map_no_dilution' and 'create_echo_transfer_list_no_dilution' functions. (Other colab TBD)
1. If neither of these assumptions fit your use case, you will need to write your own echo transfer functions using the functions in the echo_lib as examples.

----------

This colab will:
1. Take in a list of compounds at different concentrations and replicate numbers and create a platemap with wells randomly placed in the plate. 
1. Use the generated random platemap plus a source plate (provided as input) to create an transfer CSV that can be used by the Echo robot.

The colab does some validation along the way and attempts to report common errors, but be sure to spot check the output.

This colab lets you define a set of "groups" like "sample" or "control". Each group contains a set of compounds and a set of concentrations for each compound in the group. Each compound/concentration has a number of replicates within the plate.

Assumptions:
* The concentrations go out to three decimals (i.e. 0.001). 
* All the compounds within a group should be used at the same set of concentrations, with the same number of replicates per compound/concentration.
* Concentrations and volumes are unitless, but should be assumed to be the same unit throughout.

To Use This Colab:
* Adjust the Assumptions/Constants section to the set of sample wells you want to make.
* Run the colab and read all the output and validation!

Output:
* The platemap in CSV format
* The Echo transfer in CSV format




# Code Setup

In [ ]:
!git clone https://github.com/google/cell_img

In [ ]:
!pip install --quiet -e cell_img

In [ ]:
# Restart the runtime if cell_img was just pip installed or this will fail.
from cell_img.echo_robot import echo_lib

In [ ]:
import collections

import random
from copy import copy

import pandas as pd
import numpy as np
import random
import six

import matplotlib.pyplot as plt
from IPython import display

# show all the rows and columns for pandas dataframes
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
# Functions to use df_style to visualize panda dataframes
def show_map(well_df, col_to_show='well_type'):
  display.display(well_df.pivot(
        index='well_row', columns='well_col', values=col_to_show).pipe(
            df_style.discrete_colorize))
  
def show_num_map(well_df, col_to_show='well_type'):
  display.display(well_df.pivot(
        index='well_row', columns='well_col', values=col_to_show).pipe(
            df_style.linear_colorize))

# Assumptions / Constants

## Assumptions / Constants for creating random platemap

In [ ]:
# First we set up 'groups' of wells, for example 'control' and 'sample'.
# Each group will have a set of compounds and a set of concentrations. For
# each compound at each concentration, there will be a defined number of
# replicate wells to be created.
# The group, compound and concentration together define well type, for example:
# control|my_drug_1|1 
# defines the well type that is a control well, where my_drug_1
# is used at concentration 1 uM.
# Concentrations must be numbers, not strings.

# This dictionary maps which concentrations to use for each group of well
# in the output plate. 
# The string key is the name of the type of the compound, the value to the
# right is a list of concentrations for that type.
CONCENTRATIONS_PER_GROUP_MAP = {
    'control': [1, 0.1, 0.01],
    'sample': [10, 1, 0.1, 0.01],
}

# This dictionary maps which compounds to use for each group of well.
# Note that the keys for this map must match the keys for the 
# CONCENTRATIONS_PER_GROUP_MAP.
COMPOUNDS_PER_GROUP_MAP = {
    'control': ['blinded_code_1'],
    'sample': [
      'blinded_code_2',
      'blinded_code_3',
      'blinded_code_4',
      'blinded_code_5',
      'blinded_code_6',
      'blinded_code_7',
      'blinded_code_8',
      'blinded_code_9',
      'blinded_code_10',
      'blinded_code_11',
      'blinded_code_12',
      'blinded_code_13',
      'blinded_code_14',
      'blinded_code_15',
      'blinded_code_16',
      'blinded_code_17',
      'blinded_code_18',
      'blinded_code_19',
      'blinded_code_20',
      'blinded_code_21',                
    ]
}

# This the number of replicate wells for each compound at each concentration
NUM_REPLICATES_MAP = {
    ('control', 1): 10,
    ('control', 0.1): 10,
    ('control', 0.01): 10,
    ('sample', 10): 3,
    ('sample', 1): 3,
    ('sample', 0.1): 3,
    ('sample', 0.01): 3,
}

# A typical 384 well plate has rows A through P and columns 1 through 24.
# However, the wells on the outside of the plate (i.e. those in row A, row P,
# column 1, and column 24) often have the largest experimental effects, 
# likely because of evaporation during the experiment. Often, these wells are
# filled with water/DMSO and ignored. The setup below assumes this is true
# and does not include the outer wells in the list of those filled by the robot.

# If you want a different set of sample wells, you can adjust rows/cols, or
# spell them out explicitly in AVAILABLE_WELLS.
WELL_ROWS = ['B', 'C', 'D', 'E', 'F', 'G', 'H', 
             'I', 'J', 'K', 'L', 'M', 'N', 'O']
WELL_COLS = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
            13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
AVAILABLE_WELLS = []
for row in WELL_ROWS:
  for col in WELL_COLS:
    AVAILABLE_WELLS.append('%s%s' % (row, format(col, '02d')))


# These are any wells that have to be in a specific position on the plate.
# The values for each item in the list are:
# well_name, group, compound, and concentration.
# Every well_name used here should be found in the AVAILABLE_WELLS above.
NON_RANDOM_WELLS = [
   ('B05', 'my_test_well', 'blinded_code_1', 0.1),
]

# Sometimes the values defined above to do not fill all the wells avaiable.
# For example, the setup may only specify 250 randomized wells and 50
# non-random wells, out of an available 308 wells.
# In that case, this well specification will be used. 
# The values align to group name, compound, and concentration.
EXTRA_WELL_SPEC = ('dmso_control', 'DMSO', 0.0)
# Example if you want to use something other than DMSO for your extra wells,
# specific the compound and concentrations, like this:
#EXTRA_WELL_SPEC = ('empty_control', 'blinded_code_1', 0.1)

# This is the total number of well to be filled by the robot.
TOTAL_NUM_AVAILABLE_WELLS = len(AVAILABLE_WELLS)

In [ ]:
# Validate all the constants above
echo_lib.validate_platemap_setup(
    CONCENTRATIONS_PER_GROUP_MAP, COMPOUNDS_PER_GROUP_MAP, NUM_REPLICATES_MAP, 
    AVAILABLE_WELLS, NON_RANDOM_WELLS, EXTRA_WELL_SPEC)

## Assumptions / Constants for creating Echo transfer plate

In [ ]:
# Constants for constructing the Echo destination plate.
# The Echo source plate is the pure compounds used as a source by the echo.
# The Echo destination plate is the random platemap created above, with the
# compound/concentration per well as defined above.
# Note that volumes and concentrations are unitless, but assumed to be the
# same unit throughout

# The echo robot CSV file includes the barcode used for the source and
# destination plates.
ECHO_SOURCE_BARCODE = 'echo_source_barcode'
ECHO_DESTINATION_BARCODE = 'echo_dest_barcode'

# We assume we want every well in the Echo destination plate to have the 
# same volume. 
TOTAL_VOL_PER_DEST_WELL = 2
# For the Echo, if we use the compound from the source plate directly, i.e.
# do not add any DMSO to the destination well, this is the concentration of 
# compound that would result.
SOURCE_COMPOUND_CONCENTRATION = 10.

# This is the minimum volume that the echo can pipet accurately.
# Echo specification says that minimum transfer volume is 2.5 nl.
# This example colab is in ul (conversion to nl is at the final step).
# If you prefer to do all these volumes in nl, just make sure to use 
# convert_volume_unit_mulitplier = 1 in the final step to build the echo
# transfer CSV.
MIN_ECHO_PIPET = 0.00025

# So if total volume was 2 ul and the pure compound concentration was 10mM then:
# For a well that will ultimately have 10mM drug compound, we transfer 
# 2 ul of compound and do not top off with DMSO.
# For a DMSO well, we add 2 ul of DMSO.
# For a well that is 1mM, we'd add 2 ul * (1mM / 10mM) = 0.2 ul of 
# compound and then top off with enough DMSO to get to 2 ul (ie 1.8 ul).

In [ ]:
# These volumes are the WORKING volume in the source plate. 
# ** Be sure to take the dead volume of the robot into account when filling
# the source plate! ** 

# We use the string content of a CSV file here so we can avoid reading/writing
# files in this example colab. However, reading a CSV would work, replace
# this line in the cell below:
# source_plate_df = pd.read_csv(six.StringIO(SOURCE_PLATE_CSV_AS_STR))
# with code to read from a filesystem, e.g.:
# source_plate_df = pd.read_csv('/my/file/path/echo_source.csv')

# Notes: 
# The fsspec library is extremely useful if you are reading from cloud buckets
# using colab.
# One benefit of having the CSV defined in the colab is that keeping all the 
# information in one document (i.e. this colab) might make it more 
# straightforward to make sure you track exactly which source plates were
# used with which echo transfer plates in which experiment. In other words,
# help out future-you by taking making it easy for current-you to write down
# what you did.
SOURCE_PLATE_CSV_AS_STR = """compound,well,volume
blinded_code_1,A01,30
blinded_code_2,A02,30
blinded_code_3,A03,30
blinded_code_4,A04,30
blinded_code_5,A05,30
blinded_code_6,A06,30
blinded_code_7,A07,30
blinded_code_8,A08,30
blinded_code_9,A09,30
blinded_code_10,A10,30
blinded_code_11,A11,30
blinded_code_12,A12,30
blinded_code_13,A13,30
blinded_code_14,A14,30
blinded_code_15,A15,30
blinded_code_16,A16,30
blinded_code_17,A17,30
blinded_code_18,A18,30
blinded_code_19,A19,30
blinded_code_20,A20,30
blinded_code_21,A21,30
DMSO,B01,50
DMSO,B02,50
DMSO,B03,50
DMSO,B04,50
DMSO,B05,50
DMSO,B06,50
DMSO,B07,50
DMSO,B08,50
DMSO,B09,50
DMSO,B10,50
DMSO,B11,50
DMSO,B12,50
DMSO,B13,50
DMSO,B14,50
DMSO,B15,50
DMSO,B16,50
DMSO,B17,50
"""

In [ ]:
# Read the CSV string above into a dataframe
source_plate_df = pd.read_csv(six.StringIO(SOURCE_PLATE_CSV_AS_STR))

# Create Platemap

In [ ]:
#@title Build the Dictionary of Wells to Create
# Here we turn the requirements defined as constants above into a dictionary
# that describes each type of well and how many replicates of that well should
# be included in the platemap. 
wells_to_randomize = echo_lib.build_randomized_rep_dictionary(
    num_replicates_map=NUM_REPLICATES_MAP, 
    total_num_wells=TOTAL_NUM_AVAILABLE_WELLS - len(NON_RANDOM_WELLS),
    compounds_per_group_map=COMPOUNDS_PER_GROUP_MAP,
    concentrations_per_group_map=CONCENTRATIONS_PER_GROUP_MAP,
    extra_well_spec=EXTRA_WELL_SPEC)

In [ ]:
#@title Assign Wells into the Platemap
# Here we take the set of wells we want to include, and we randomly assign them
# into the wells. 
platemap_df = echo_lib.build_platemap(
    wells_to_randomize, NON_RANDOM_WELLS,
    AVAILABLE_WELLS)

In [ ]:
# Print out all the number of wells per group
platemap_df.group.value_counts()

In [ ]:
# Validate that the platemap has the columns we expect. This doesn't print all
# the columns, but enough to look at them and see that they make sense.
# I use "sample(10)" here to randomly pull some of the rows from the dataframe so
# I can do a spot check.
platemap_df.sample(10)

In [ ]:
#@title Spot check: Pick a random type and check it has the right number of replicates.

# (A different random sample will be picked each time you run this cell.
# Run it multiple times to see different results.)

# randomly pick a well type (well type = type + compound + concentration)
random_compound_group = random.sample(COMPOUNDS_PER_GROUP_MAP.keys(), 1)[0]
random_compound = random.sample(COMPOUNDS_PER_GROUP_MAP[random_compound_group], 1)[0]
random_concentration = random.sample(CONCENTRATIONS_PER_GROUP_MAP[random_compound_group], 1)[0]
random_well_type = echo_lib.build_well_type(random_compound_group, random_compound, random_concentration)
expected_num_replicates = NUM_REPLICATES_MAP[random_compound_group, random_concentration]
actual_num_replicates = len(platemap_df.query('well_type == "%s"' % random_well_type))

# Check that there are the expected number of wells
if expected_num_replicates != actual_num_replicates:
  print('**ERROR! Expected %d replicates for type %s, but found %d' % (
      expected_num_replicates, random_well_type, actual_num_replicates))
else:
  print('Spot checked "%s" and found %d wells, as expected.\n' % (
      random_well_type, actual_num_replicates))

# and show the wells 
platemap_df.query('well_type == "%s"' % random_well_type)

In [ ]:
#@title Visualize the randomized platemap
# Visualize the groups around the platemap
show_map(platemap_df, 'group')

In [ ]:
# Visualize the compounds around the platemap
show_map(platemap_df, 'compound')

In [ ]:
# Visualize the concentrations in the randomized platemap
show_num_map(platemap_df, 'concentration')

In [ ]:
# and, finally, all the different well_types
show_map(platemap_df, 'well_type')

In [ ]:
# Prints the CSV in the right format to copy & paste into a CSV.
print(platemap_df.to_csv(index=False))

# Create Echo Transfer CSV

In [ ]:
# The volumes in the source plate are left as global variabless 
# to make it easy to build multiple destination
# plates from a single source plate if you want to add that functionality.
volume_used_per_source_well_map = collections.defaultdict(int)
volume_used_per_compound_map = collections.defaultdict(int)

echo_source_map, volume_remaining_per_source_well_map = echo_lib.build_source_map(source_plate_df)

In [ ]:
transfer_list_1, volume_per_dest_well_map_1 = echo_lib.create_echo_transfer_list(
              source_plate_barcode=ECHO_SOURCE_BARCODE, 
              dest_plate_barcode=ECHO_DESTINATION_BARCODE,
              source_map=echo_source_map, 
              well_df=platemap_df,
              volume_per_compound_map=volume_used_per_compound_map, 
              volume_remaining_per_source_well_map=volume_remaining_per_source_well_map, 
              volume_used_per_source_well_map=volume_used_per_source_well_map,
              total_vol_per_dest_well=TOTAL_VOL_PER_DEST_WELL,
              source_compound_concentration=SOURCE_COMPOUND_CONCENTRATION, 
              min_pipet_volume=MIN_ECHO_PIPET)

transfer_list_1[:3]

In [ ]:
print('The max transfer volume is %.4f ul' % (max([x[4] for x in transfer_list_1])))
print('The min transfer volume is %.4f ul' % (min([x[4] for x in transfer_list_1])))

In [ ]:
for k,v in volume_used_per_compound_map.items():
  print('%s\t\t%.2f' % (k, v))

In [ ]:
print(echo_lib.build_echo_transfer_str(
    transfer_list_1, convert_volume_unit_mulitplier=1000))